In [1]:
import cv2

In [2]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd

In [559]:
%qtconsole

In [4]:
# list files in directory

In [3]:
os.chdir("/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaFruits_28Sept2016/")

In [4]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [5]:
# find bad contours
def is_contour_good(c, img):
    # approximate the contour
    area1 = cv2.contourArea(c, False) 

    # the contour is 'bad' if it is really small (less then 1000 px)
    # the contour is bad, if it is on the border
    x,y,w,h = cv2.boundingRect(c)       
    if x <= 1 or y <=1 or x+w+1 >= img.shape[1] or y+h+1 >= img.shape[0] or area1 < 1000:
        return False              
    else:
        return True 

In [6]:
photoList = [ x for x in onlyfiles if "DSC" not in x and not x.startswith('.') and x.endswith('.jpg')]

In [7]:
plants = [v.translate(None, '.jpg') for v in photoList]

In [8]:
def kalmSeg(kk):
    # read in image as greyscale
    #kk = 70
    #kk += 1
    fpth = photoList[kk]
    img = cv2.imread(fpth,0)

    # plt.clf()
    # plt.xlim(6000, 0)
    # plt.ylim(0, 4000)
    # io.imshow(img) # so fast, compared to sklearn

    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(img,(5,5),0)
    # ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    ret,th3 = cv2.threshold(blur,125,255,cv2.THRESH_BINARY_INV)
    # I manually chose a threshold of 125 above



    # now get rid of the leafy bits
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    erosion = cv2.erode(th3,kernel,iterations = 10)
    #io.imshow(erosion)

    dilation = cv2.dilate(erosion, kernel, iterations = 10)
    #plt.imshow(dilation, cmap = 'gray')

    # plt.xlim(6000, 0)
    # plt.ylim(0, 4000)
    # io.imshow(dilation)

    # close small holes
    image = dilation.copy()
    #io.imshow(image)

    (cnts, _) = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.ones(image.shape[:2], dtype="uint8") * 0 # create a blank black mask

    # loop over the contours
    for c in cnts:
        # if the contour is good, draw it on the mask (draw white on a black background)
        if is_contour_good(c, image):
            cv2.drawContours(mask, [c], -1, 255, -1)

    # see if any bad contours were found
    filHoles = 'no holes were filled--' + str((mask == image).all()) # will be false if a small hole was filled

    # remove the contours from the image and show the resulting images
    image2 = mask.copy()
    #io.imshow( mask) # mask

    #io.imshow( image) # after

    # find contours for fruits
    cts, im2 = cv2.findContours(image2.copy(),  cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    perimeters = [cv2.arcLength(cts[ii], True) for ii in range(len(cts))]

    # calculate areas
    areas = [cv2.contourArea(cts[ii], False) for ii in range(len(cts))]

    # calculate circularity
    # circularity = (perimeters .^ 2) ./ (4 * pi * area)
    # If this is closer to 1, then the object is more circle-like
    circularity = [(x **2) / (4*3.14159*y) for x,y in zip(perimeters, areas)]


    arrs = np.array([i < 2 for i in circularity]) # get only roughly circular objects
    # circles will be close to 0


    ctd = np.array(cts)
    filt_contours = ctd[arrs] # will let me see if we miss any fruits




    # # draw filtered contours (white)
    # plt.clf()
    # plt.xlim(6000, 0)
    # plt.ylim(0, 4000)
    cv2.drawContours(img, filt_contours, -1, (255,255,0), -1) # note, this modifies the original image, "img"
    # io.imshow(img)

    # calculate diameters in pixels
    # area = pi * r ^2
    # r = sqrt(area / pi)

    radii = np.array([np.sqrt(aa / 3.14159) for aa in areas])

    # convert to mm instead of pixels
    dia_mm = np.array([aa2/max(radii) * 2 * 10 for aa2 in radii])


    # show circle with largest diameter (black)
    bgCirc = np.array(radii == radii.max())

    # draw filtered contours (black)
    cv2.drawContours(img, ctd[bgCirc], -1, (0,255,0), -1) # note, this modifies the original image, "img"
    #io.imshow(img)

    # compute the center of the contour
    Moms = np.array([cv2.moments(cc) for cc in cts])
    # get xy coordinates
    cs = np.array([[int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])] for M in Moms])


    xx = np.array([cs[ii, 0] for ii in range(len(cs))])
    yy = np.array([cs[ii, 1] for ii in range(len(cs))])

    plt.clf()
    io.imshow(img)

    plt.plot(xx, yy, 'rp')
    plt.xlim(6000, 0)
    plt.ylim(0, 4000)

    # label diameters
    labels = ['dia_{0}'.format(round(i, 2)) for i in dia_mm]
    for label, x, y in zip(labels, xx, yy):
        plt.annotate(
            label, color = 'grey', size = 10,
            xy = (x, y), xytext = (0, 0),
            textcoords = 'offset points', ha = 'right', va = 'bottom')



    plt.show()

    # save plot, but make new directory if it doesn't already exist
    if not os.path.isdir("SegmentedImages/"):
        os.mkdir("SegmentedImages/")
        
    plt.savefig('SegmentedImages/' + str(plants[kk]) + '_segmented.png')

    print len(xx) # number of fruits (including big circle)
    print filHoles
    print 'kk = ' + str(kk)
    
    # save .csv file with information
    # return a pandas data frame
    tmp_DF = pd.DataFrame({'areas': areas, 
              'perimeters': perimeters, 
              'dia_mm': np.ndarray.tolist(dia_mm), 
             'plantNum': plants[kk]})
    return tmp_DF
    
    

In [10]:
# create empty data frame for saving image data
df_final = pd.DataFrame({'areas': [], 
              'perimeters': [], 
              'dia_mm': [], 
             'plantNum': []})

#loop through all and save images
for yy in range(len(plants)):
    tmp_DF = kalmSeg(yy)
    
    #append to final data frame
    df_final = df_final.append(tmp_DF)
    

8
no holes were filled--True
kk = 0
25
no holes were filled--True
kk = 1
33
no holes were filled--False
kk = 2
31
no holes were filled--True
kk = 3
22
no holes were filled--True
kk = 4
9
no holes were filled--True
kk = 5
9
no holes were filled--True
kk = 6
13
no holes were filled--True
kk = 7
9
no holes were filled--True
kk = 8
12
no holes were filled--True
kk = 9
13
no holes were filled--True
kk = 10
43
no holes were filled--True
kk = 11
16
no holes were filled--True
kk = 12
17
no holes were filled--True
kk = 13
5
no holes were filled--True
kk = 14
26
no holes were filled--True
kk = 15
15
no holes were filled--True
kk = 16
3
no holes were filled--True
kk = 17
5
no holes were filled--True
kk = 18
15
no holes were filled--True
kk = 19
31
no holes were filled--True
kk = 20
18
no holes were filled--False
kk = 21
10
no holes were filled--True
kk = 22
13
no holes were filled--True
kk = 23
9
no holes were filled--True
kk = 24
36
no holes were filled--True
kk = 25
31
no holes were filled--Tru

In [11]:
# write the data to a file
df_final.to_csv('SegmentedImages/kalmiaFruitFinal.csv')

In [538]:
plants[kk]

'1129_74_A__5'

In [553]:
df_final = pd.DataFrame({'areas': [], 
              'perimeters': [], 
              'dia_mm': [], 
             'plantNum': []})

tmp_DF = pd.DataFrame({'areas': areas, 
              'perimeters': perimeters, 
              'dia_mm': np.ndarray.tolist(dia_mm), 
             'plantNum': plants[kk]})


df_final = df_final.append(tmp_DF)

In [416]:
# label diameters on plot

plt.clf()
io.imshow(img)

plt.plot(xx, yy, 'rp')
plt.xlim(6000, 0)
plt.ylim(0, 4000)

labels = ['dia_{0}'.format(round(i, 2)) for i in dia_mm]
for label, x, y in zip(labels, xx, yy):
    plt.annotate(
        label, color = 'grey', size = 10,
        xy = (x, y), xytext = (0, 0),
        textcoords = 'offset points', ha = 'right', va = 'bottom')
    
plt.show()

In [483]:
# label points on plot

plt.clf()
io.imshow(img)

plt.plot(xx, yy, 'rp')
plt.xlim(6000, 0)
plt.ylim(0, 4000)

labels = ['point{0}'.format(i) for i in range(len(xx))]
for label, x, y in zip(labels, xx, yy):
    plt.annotate(
        label, color = 'grey', size = 10,
        xy = (x, y), xytext = (0, 0),
        textcoords = 'offset points', ha = 'right', va = 'bottom')
    
plt.show()